# Assumptions

## 1. There are no measurement errors.

If we had to account for measurement errors then we would have to run multiple surface codes cycles to infer where the error has occured. There are efficient methods that work for sparse errors but if the error probabilities are high then surface codes cannot correct the errors.

# Imports

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
import numpy as np
import matplotlib.pyplot as plt
from qiskit.result import marginal_counts

# Parameters

In [2]:
surface_code_sq = 5

In [3]:
if surface_code_sq%2 ==0:
    raise Exception('surface_code_sq should be odd')

In [4]:
# n,k,d
n = (surface_code_sq**2)//2 + 1

# Logical Qubit

In [5]:
data_qubits = QuantumRegister(n, 'd')
ancilla_qubits_1 = QuantumRegister((n-1)//2, 'a1')
ancilla_qubits_2 = QuantumRegister((n-1)//2, 'a2')
syndrome_cregs_1 = ClassicalRegister((n-1)//2, 's1')
syndrome_cregs_2 = ClassicalRegister((n-1)//2, 's2')

In [6]:
data_qubits_index = 0
ancilla_qubits_1_index = 0
ancilla_qubits_2_index = 0

logical_qubit_array = []

for i in range(surface_code_sq):
    if i%2 == 0:
        for j in range(surface_code_sq//2):
            logical_qubit_array.append(data_qubits[data_qubits_index])
            data_qubits_index += 1
            logical_qubit_array.append(ancilla_qubits_1[ancilla_qubits_1_index])
            ancilla_qubits_1_index += 1
        logical_qubit_array.append(data_qubits[data_qubits_index])
        data_qubits_index += 1
    else:
        for j in range(surface_code_sq//2):
            logical_qubit_array.append(ancilla_qubits_2[ancilla_qubits_2_index])
            ancilla_qubits_2_index += 1
            logical_qubit_array.append(data_qubits[data_qubits_index])
            data_qubits_index += 1
        logical_qubit_array.append(ancilla_qubits_2[ancilla_qubits_2_index])
        ancilla_qubits_2_index += 1

In [7]:
logical_qubit_array = np.array(logical_qubit_array, dtype=QuantumRegister)

In [8]:
logical_qubit_2d = logical_qubit_array.reshape(surface_code_sq, surface_code_sq)

In [9]:
ancilla_qubits_1_neighbours = { }
ancilla_qubits_2_neighbours = { }

for row_index in range(0, surface_code_sq, 2):
    for col_index in range(1, surface_code_sq, 2):
        ancilla = logical_qubit_2d[row_index, col_index]
        neighbours = []
        for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
            data_qubit_row_index = row_index + i
            data_qubit_col_index = col_index + j
            if data_qubit_row_index > -1 and data_qubit_col_index > -1 and data_qubit_row_index < surface_code_sq and data_qubit_col_index < surface_code_sq:
                data_qubit = logical_qubit_2d[data_qubit_row_index, data_qubit_col_index]
                neighbours.append(data_qubit)
        ancilla_qubits_1_neighbours[ancilla] = neighbours

for row_index in range(1, surface_code_sq, 2):
    for col_index in range(0, surface_code_sq, 2):
        ancilla = logical_qubit_2d[row_index, col_index]
        neighbours = []
        for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
            data_qubit_row_index = row_index + i
            data_qubit_col_index = col_index + j
            if data_qubit_row_index > -1 and data_qubit_col_index > -1 and data_qubit_row_index < surface_code_sq and data_qubit_col_index < surface_code_sq:
                data_qubit = logical_qubit_2d[data_qubit_row_index, data_qubit_col_index]
                neighbours.append(data_qubit)
        ancilla_qubits_2_neighbours[ancilla] = neighbours
                

We no longer need the 2d array `logical_qubit_2d` because we created the neighbours dictionaries

In [10]:
ancilla_qubits_2_neighbours[ancilla_qubits_2[0]]

[Qubit(QuantumRegister(13, 'd'), 0),
 Qubit(QuantumRegister(13, 'd'), 3),
 Qubit(QuantumRegister(13, 'd'), 5)]

In [11]:
del logical_qubit_2d

In [12]:
backend = Aer.get_backend('qasm_simulator')

# Test circ

In [13]:
def surface_code_encode(qc: QuantumCircuit,
                        x_ancilla_qubits_neighbours: dict[QuantumRegister, list[QuantumRegister]],
                        z_ancilla_qubits_neighbours: dict[QuantumRegister, list[QuantumRegister]]):
    qc.h(list(x_ancilla_qubits_neighbours.keys()))
    qc.barrier()
    for z_ancilla, neighbour_data_qubits in z_ancilla_qubits_neighbours.items():
        for neighbour_data_qubit in neighbour_data_qubits:
            qc.cx(neighbour_data_qubit, z_ancilla)
        qc.barrier()

    for x_ancilla, neighbour_data_qubits in x_ancilla_qubits_neighbours.items():
        for neighbour_data_qubit in neighbour_data_qubits:
            qc.cx(x_ancilla, neighbour_data_qubit)
        qc.barrier()
    

In [31]:
def surface_code_measure_syndrome(qc: QuantumCircuit, x_ancilla_qubits: QuantumRegister, z_ancilla_qubits: QuantumRegister,
                                  x_syndrome_cregs: ClassicalRegister, z_syndrome_cregs: ClassicalRegister):
    qc.barrier()
    qc.h(x_ancilla_qubits)
    qc.barrier()
    qc.measure(x_ancilla_qubits, x_syndrome_cregs)
    qc.measure(z_ancilla_qubits, z_syndrome_cregs)


In [32]:
# qc = QuantumCircuit(data_qubits, ancilla_qubits_1, ancilla_qubits_2, syndrome_cregs_1, syndrome_cregs_2)
# qc.h(data_qubits)
# surface_code_encode(qc=qc,
#                     x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
#                     z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)
# surface_code_measure_syndrome(qc=qc,
#                               x_ancilla_qubits=ancilla_qubits_1,
#                               z_ancilla_qubits=ancilla_qubits_2,
#                               x_syndrome_cregs=syndrome_cregs_1,
#                               z_syndrome_cregs=syndrome_cregs_2)

# qc.draw('mpl')

In [33]:
ancilla_qubits_2_neighbours[ancilla_qubits_2[0]]

[Qubit(QuantumRegister(13, 'd'), 0),
 Qubit(QuantumRegister(13, 'd'), 3),
 Qubit(QuantumRegister(13, 'd'), 5)]

In [34]:
ancilla_qubits_2_neighbours[ancilla_qubits_2[0]]

[Qubit(QuantumRegister(13, 'd'), 0),
 Qubit(QuantumRegister(13, 'd'), 3),
 Qubit(QuantumRegister(13, 'd'), 5)]

In [48]:
syndrome_cregs_1 = ClassicalRegister((n-1)//2, 's1')
syndrome_cregs_2 = ClassicalRegister((n-1)//2, 's2')
syndrome_cregs_3 = ClassicalRegister((n-1)//2, 's3')
syndrome_cregs_4 = ClassicalRegister((n-1)//2, 's4')
syndrome_cregs_5 = ClassicalRegister((n-1)//2, 's5')
syndrome_cregs_6 = ClassicalRegister((n-1)//2, 's6')
syndrome_cregs_7 = ClassicalRegister((n-1)//2, 's7')
syndrome_cregs_8 = ClassicalRegister((n-1)//2, 's8')


qc = QuantumCircuit(data_qubits, ancilla_qubits_1, ancilla_qubits_2, syndrome_cregs_1, syndrome_cregs_2, syndrome_cregs_3, syndrome_cregs_4,
                    syndrome_cregs_5, syndrome_cregs_6, syndrome_cregs_7, syndrome_cregs_8)
qc.h(data_qubits)

surface_code_encode(qc=qc,
                    x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
                    z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)

surface_code_measure_syndrome(qc=qc,
                              x_ancilla_qubits=ancilla_qubits_1,
                              z_ancilla_qubits=ancilla_qubits_2,
                              x_syndrome_cregs=syndrome_cregs_1,
                              z_syndrome_cregs=syndrome_cregs_2)

surface_code_encode(qc=qc,
                    x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
                    z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)

for i, j in zip(ancilla_qubits_2, syndrome_cregs_2):
    qc.x(i).c_if(j, 1)

surface_code_measure_syndrome(qc=qc,
                              x_ancilla_qubits=ancilla_qubits_1,
                              z_ancilla_qubits=ancilla_qubits_2,
                              x_syndrome_cregs=syndrome_cregs_3,
                              z_syndrome_cregs=syndrome_cregs_4)

# qc.z(data_qubits[3])
# qc.x(data_qubits[3])

surface_code_encode(qc=qc,
                    x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
                    z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)

for i, j in zip(ancilla_qubits_2, syndrome_cregs_2):
    qc.x(i).c_if(j, 1)

surface_code_measure_syndrome(qc=qc,
                              x_ancilla_qubits=ancilla_qubits_1,
                              z_ancilla_qubits=ancilla_qubits_2,
                              x_syndrome_cregs=syndrome_cregs_5,
                              z_syndrome_cregs=syndrome_cregs_6)

surface_code_encode(qc=qc,
                    x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
                    z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)

for i, j in zip(ancilla_qubits_2, syndrome_cregs_2):
    qc.x(i).c_if(j, 1)

surface_code_measure_syndrome(qc=qc,
                              x_ancilla_qubits=ancilla_qubits_1,
                              z_ancilla_qubits=ancilla_qubits_2,
                              x_syndrome_cregs=syndrome_cregs_7,
                              z_syndrome_cregs=syndrome_cregs_8)

job = backend.run(qc, shots=10)
counts = job.result().get_counts()

counts

{'100000 000000 100000 000000 100000 000000 100000 000000': 1,
 '111110 000000 111110 000000 111110 000000 111110 000000': 1,
 '110110 000000 110110 000000 110110 000000 110110 000000': 1,
 '110100 000000 110100 000000 110100 000000 110100 000000': 1,
 '100001 000000 100001 000000 100001 000000 100001 000000': 1,
 '010101 000000 010101 000000 010101 000000 010101 000000': 1,
 '010110 000000 010110 000000 010110 000000 010110 000000': 1,
 '011101 000000 011101 000000 011101 000000 011101 000000': 1,
 '001011 000000 001011 000000 001011 000000 001011 000000': 1,
 '000010 000000 000010 000000 000010 000000 000010 000000': 1}

In [19]:
# marginal_counts(counts, range(12, 36))

In [20]:
# syndrome_cregs_1 = ClassicalRegister((n-1)//2, 's1')
# syndrome_cregs_2 = ClassicalRegister((n-1)//2, 's2')
# syndrome_cregs_3 = ClassicalRegister((n-1)//2, 's3')
# syndrome_cregs_4 = ClassicalRegister((n-1)//2, 's4')
# syndrome_cregs_5 = ClassicalRegister((n-1)//2, 's5')
# syndrome_cregs_6 = ClassicalRegister((n-1)//2, 's6')


# qc = QuantumCircuit(data_qubits, ancilla_qubits_1, ancilla_qubits_2, syndrome_cregs_1, syndrome_cregs_2, syndrome_cregs_3, syndrome_cregs_4,
#                     syndrome_cregs_5, syndrome_cregs_6)
# qc.h(data_qubits)

# surface_code_encode(qc=qc,
#                     x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
#                     z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)

# surface_code_measure_syndrome(qc=qc,
#                               x_ancilla_qubits=ancilla_qubits_1,
#                               z_ancilla_qubits=ancilla_qubits_2,
#                               x_syndrome_cregs=syndrome_cregs_1,
#                               z_syndrome_cregs=syndrome_cregs_2)

# # qc.x(data_qubits[0]).c_if(syndrome_cregs_2[0], 1)

# # surface_code_encode(qc=qc,
# #                     x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
# #                     z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)
# # surface_code_measure_syndrome(qc=qc,
# #                               x_ancilla_qubits=ancilla_qubits_1,
# #                               z_ancilla_qubits=ancilla_qubits_2,
# #                               x_syndrome_cregs=syndrome_cregs_3,
# #                               z_syndrome_cregs=syndrome_cregs_4)

# # surface_code_encode(qc=qc,
# #                     x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
# #                     z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)
# # surface_code_measure_syndrome(qc=qc,
# #                               x_ancilla_qubits=ancilla_qubits_1,
# #                               z_ancilla_qubits=ancilla_qubits_2,
# #                               x_syndrome_cregs=syndrome_cregs_5,
# #                               z_syndrome_cregs=syndrome_cregs_6)

# job = backend.run(qc, shots=1024)
# counts = job.result().get_counts()

# len(counts)